In [1]:
import numpy as np
import matplotlib.pyplot as plt
# import matplotlib.colors as mcolors
import pickle
import tikzplotlib
def tikzplotlib_fix_ncols(obj):
    """
    workaround for matplotlib 3.6 renamed legend's _ncol to _ncols, which breaks tikzplotlib
    """
    if hasattr(obj, "_ncols"):
        obj._ncol = obj._ncols
    for child in obj.get_children():
        tikzplotlib_fix_ncols(child)

In [2]:
def get_data(ep, num_qubits, num_meas, method, noise="None", decode=True):
    degree=num_meas
    num_reservoirs=20
    timeplex=10
    string_identifier=str(ep)+"_casename"+str(casename)+"_num_qubits"+str(num_qubits)+"_num_meas"+str(num_meas)
    string_identifier+="_degree"+str(degree)+"_num_reservoirs"+str(num_reservoirs)+"_timeplex"+str(timeplex)
    string_identifier+="_method"+str(method)+"_noise"+str(noise)
    if not decode:
        string_identifier+="_decodeFalse"
    string_identifier+=".pickle"
    name="X_train"+string_identifier
    # print("name=",name)
    with open(name,"rb") as f:
        X_train = pickle.load(f)
    
    name="X_test"+string_identifier
    with open(name,"rb") as f:
        X_test = pickle.load(f)
    
    name="score"+string_identifier
    with open(name,"rb") as f:
        score = pickle.load(f)
        
    name="prediction"+string_identifier
    with open(name,"rb") as f:
        prediction = pickle.load(f)
        
    name="state"+string_identifier
    with open(name,"rb") as f:
        state = pickle.load(f)
    return X_train, X_test, score, prediction, state, string_identifier

In [3]:
from reservoirpy.datasets import logistic_map
from run_reservoir import henon1d

Plot convergence with training

In [4]:
casename="logistic"
ts=logistic_map(100000, r=3.9, x0=0.5).flatten()
lines=[]
for nm in [[3,2], [4,3], [4,2], [5,4], [5,3], [5,2]]:
    tmp="\midrule \\multicolumn{7}{c}{"+str(nm[0])+" qubits, measure "+str(nm[1])+" stabilizers } \\\\ \midrule"
    lines.append(tmp)
    for ep in range(5):
        # plt.figure()
        for method in ["classical", "quantum_part", "quantum_stab"]:
            X_train, X_test, score, prediction, state, basen = get_data(ep, nm[0], nm[1], method, "None")
            if method=="classical":
                linestr=str(len(X_train))+" & "
            r=3.9
            tmp=prediction[len(X_train):]
            if casename=="logistic":
                tmp2=(tmp[1:] - r*tmp[:-1]*(1-tmp[:-1]))**2
            elif casename=="henon":
                a=1.4
                b=0.3
                tmp2=(tmp[2:] - (1 - a*tmp[1:-1]**2 + b*tmp[:-2]))**2
            # if not method=="classical":
            #     plt.plot(tmp2, label=str(ep))
            val = np.sqrt(np.sum(tmp2))
            if val<1:
                linestr+="\\cellcolor{lightgray}"
            linestr+="{:.2e}".format(val)
            linestr+=" & "
        lines.append(linestr)
        # plt.legend()
for tmp in lines:
    print(tmp)

\midrule \multicolumn{7}{c}{3 qubits, measure 2 stabilizers } \\ \midrule
35 & 1.12e+00 & \cellcolor{lightgray}7.27e-01 & \cellcolor{lightgray}5.52e-01 & 
68 & \cellcolor{lightgray}4.78e-01 & \cellcolor{lightgray}9.75e-01 & \cellcolor{lightgray}4.95e-01 & 
101 & 7.34e+02 & \cellcolor{lightgray}3.09e-01 & \cellcolor{lightgray}2.91e-01 & 
134 & 6.73e+02 & \cellcolor{lightgray}1.95e-01 & \cellcolor{lightgray}2.27e-01 & 
167 & \cellcolor{lightgray}4.73e-02 & \cellcolor{lightgray}2.05e-01 & \cellcolor{lightgray}2.78e-01 & 
\midrule \multicolumn{7}{c}{4 qubits, measure 3 stabilizers } \\ \midrule
35 & \cellcolor{lightgray}6.94e-01 & 1.40e+00 & \cellcolor{lightgray}2.34e-01 & 
68 & \cellcolor{lightgray}6.01e-02 & \cellcolor{lightgray}3.54e-01 & \cellcolor{lightgray}2.50e-01 & 
101 & 4.18e+02 & \cellcolor{lightgray}1.86e-01 & \cellcolor{lightgray}1.43e-01 & 
134 & \cellcolor{lightgray}2.66e-02 & \cellcolor{lightgray}1.12e-01 & \cellcolor{lightgray}9.50e-02 & 
167 & 2.81e+03 & \cellcolor{lightg

In [5]:
casename="henon"
ts = henon1d(10000)
ind=-1
for nm in [[3,2], [4,3], [4,2], [5,4], [5,3], [5,2]]:
    ind+=1
    for ep in range(5):
        # plt.figure()
        linestr=""
        for method in ["classical", "quantum_part", "quantum_stab"]:
            X_train, X_test, score, prediction, state, basen = get_data(ep, nm[0], nm[1], method, "None")
            r=3.9
            tmp=prediction[len(X_train):]
            if casename=="logistic":
                tmp2=(tmp[1:] - r*tmp[:-1]*(1-tmp[:-1]))**2
            elif casename=="henon":
                a=1.4
                b=0.3
                tmp2=(tmp[2:] - (1 - a*tmp[1:-1]**2 + b*tmp[:-2]))**2
            # if not method=="classical":
            #     plt.plot(tmp2, label=str(ep))
            val = np.sqrt(np.sum(tmp2))
            if val<1:
                linestr+="\\cellcolor{lightgray}"
            linestr+="{:.2e}".format(val)
            if method=="quantum_stab":
                linestr+=" \\\\"
            else:
                linestr+=" & "
        ind+=1
        lines[ind]+=linestr
        # plt.legend()
for tmp in lines:
    print(tmp)

\midrule \multicolumn{7}{c}{3 qubits, measure 2 stabilizers } \\ \midrule
35 & 1.12e+00 & \cellcolor{lightgray}7.27e-01 & \cellcolor{lightgray}5.52e-01 & 5.71e+00 & 1.35e+00 & 1.50e+00 \\
68 & \cellcolor{lightgray}4.78e-01 & \cellcolor{lightgray}9.75e-01 & \cellcolor{lightgray}4.95e-01 & 1.04e+02 & \cellcolor{lightgray}9.89e-01 & 1.27e+00 \\
101 & 7.34e+02 & \cellcolor{lightgray}3.09e-01 & \cellcolor{lightgray}2.91e-01 & 1.77e+03 & \cellcolor{lightgray}3.70e-01 & \cellcolor{lightgray}6.00e-01 \\
134 & 6.73e+02 & \cellcolor{lightgray}1.95e-01 & \cellcolor{lightgray}2.27e-01 & 1.02e+03 & \cellcolor{lightgray}6.64e-01 & \cellcolor{lightgray}2.57e-01 \\
167 & \cellcolor{lightgray}4.73e-02 & \cellcolor{lightgray}2.05e-01 & \cellcolor{lightgray}2.78e-01 & 9.97e+03 & \cellcolor{lightgray}3.34e-01 & \cellcolor{lightgray}5.05e-01 \\
\midrule \multicolumn{7}{c}{4 qubits, measure 3 stabilizers } \\ \midrule
35 & \cellcolor{lightgray}6.94e-01 & 1.40e+00 & \cellcolor{lightgray}2.34e-01 & 7.82e+00 &